<a href="https://colab.research.google.com/github/davidgoins236/data_viz_final_project/blob/main/data_viz_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Title: Visual History of Formula 1 Racing: An Exploratory Data Analysis

Author: David Goins

Goal: Understand how Formula 1 has evolved over time using historical data with a seconary objective to evaluate which visualizations communicate these trends most effectively.

In [1]:
!git clone https://github.com/davidgoins236/data_viz_final_project
%cd data_viz_final_project

fatal: destination path 'data_viz_final_project' already exists and is not an empty directory.
/content/data_viz_final_project


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import altair as alt
#required to save ploty charts as static images to view in github
from IPython.display import Image
alt.renderers.enable('default')  # makes sure PNG is used in static output

RendererRegistry.enable('default')

# Functions:
- For this project I leveraged some custom functions in scenarios where I wanted to explore

# Data Load, Wrangling & Cleaning

In [3]:
circuits = pd.read_csv('circuits.csv')
constructor_results = pd.read_csv('constructor_results.csv')
constructor_standings = pd.read_csv('constructor_standings.csv')
constructors = pd.read_csv('constructors.csv')
driver_standings = pd.read_csv('driver_standings.csv')
drivers = pd.read_csv('drivers.csv')
lap_times = pd.read_csv('lap_times.csv')
pit_stops = pd.read_csv('pit_stops.csv')
quali = pd.read_csv('qualifying.csv')
races = pd.read_csv('races.csv')
results = pd.read_csv('results.csv')
seasons = pd.read_csv('seasons.csv')
sprint = pd.read_csv('sprint_results.csv')
status = pd.read_csv('status.csv')

In [4]:
combined_results_df = results.merge(races, how = 'left', on = 'raceId',suffixes=['_result','_race'])\
    .merge(drivers, how = 'left', on = 'driverId',suffixes=['','_driver'])\
    .merge(constructors, how = 'left', on = 'constructorId',suffixes=['','_constructor'])\
    .merge(circuits, how = 'left', on = 'circuitId',suffixes=['','_circuit'])

combined_results_df.describe()

,resultId,raceId,driverId,constructorId,grid,positionOrder,points,laps,statusId,year,round,circuitId,lat,lng,alt
count,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000,26759.000000
mean,13380.977391,551.687283,278.673530,50.180537,11.134796,12.794051,1.987632,46.301768,17.224971,1991.394372,8.511192,23.820808,33.949556,5.724416,276.847416
std,7726.134642,313.265036,282.703039,61.551498,7.202860,7.665951,4.351209,29.496557,26.026104,19.952885,5.070231,19.112002,25.246080,57.632776,409.086550
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1950.000000,1.000000,1.000000,-37.849700,-118.189000,-7.000000
25%,6690.500000,300.000000,57.000000,6.000000,5.000000,6.000000,0.000000,23.000000,1.000000,1977.000000,4.000000,9.000000,34.843100,-1.016940,18.000000
50%,13380.000000,531.000000,172.000000,25.000000,11.000000,12.000000,0.000000,53.000000,10.000000,1991.000000,8.000000,18.000000,43.734700,6.947500,153.000000
75%,20069.500000,811.000000,399.500000,63.000000,17.000000,18.000000,2.000000,66.000000,14.000000,2009.000000,12.000000,34.000000,49.254200,14.764700,401.000000
max,26764.000000,1144.000000,862.000000,215.000000,34.000000,39.000000,50.000000,200.000000,141.000000,2024.000000,24.000000,80.000000,57.265300,144.968000,2227.000000


Resable Objects

In [5]:
f1_color_palette = {
    "F1 Red": "#FF1801",           # Primary F1 brand red
    "Black": "#000000",            # Used in logo and typography
    "White": "#FFFFFF",            # Used for contrast, clean background
    "Charcoal Gray": "#1E1E1E",    # Background in broadcast graphics
    "Silver Gray": "#A3A3A3",      # For grid lines, secondary text
    "Soft Red": "#FF5C5C",         # Highlight or hover red
    "Dark Gray": "#2E2E2E",        # UI panels and dashboard elements
    "Track Asphalt": "#3A3A3C",    # Often used for circuit maps
    "Pirelli Yellow": "#FFD700",   # Pirelli tire branding
    "DRS Green": "#27AE60",        # Used to indicate DRS zones
}


# Evolution of the Sport Over Time
 - n of Races per Season
 - Circuit Debuts per Season
 - n of Teams per Season
 - Fastest Lap Times per Track Through Time
 - Pit Stop Speed over Time

## Races per Season:

This time series plot is designed to show the expansion of the number of races over the history of the sport. Some key events are flagged using verticle lines that attempt to provide context for changes in trend. For example, the 2020 season was an obvious devaition from trend due to COVID 19. Overall, the plot shows rapid expansion from 1950 to 1980 followed by a period of stability that resulted from the consolidation of governance in the 80s. Expansion began again in the 2000s as F1 actively expanded to new markets in Asia and the Middle East (e.g., first Chinese Grand Prix in 2004, Singapore 2008).

In [16]:
#note: for this project I use plt_df as a temporary dataframe that is specific to the cell in which it was run
#this convention allows vis-specific manipulation without impacting broad use dataframes
plt_df = races.groupby(['year']).agg(race_count = ('raceId', 'count')).reset_index()
fig = px.line(plt_df, x = 'year', y = 'race_count', title = 'Number of Races per Season',markers='.',
              height =500,width=1000,template='seaborn')
#clean up the axis labels
fig.update_layout(xaxis_title = 'Season', yaxis_title = 'Number of Races')

#flag some relevant events from wikipeida: https://en.wikipedia.org/wiki/Formula_One

key_events = {
    1950: "FIA Established",
    1971: "Formation of FOCA",
    1981: "Concorde Agreement: Governance",
    2000: "Effort to Expand to New Markets",
    2020: "Races Canceled Due to Covid 19"
}
for year, event in key_events.items():
    fig.add_vline(
        x=year,
        annotation_text = event,
        opacity=1,
        annotation_textangle=45,
        line_width=2,
        line_dash="dash",
        line_color="grey",
    )
fig.write_html("n_races.html")
fig.show()

## New Circuit Count per Season

This view shows that expansion has been cautious and incremental. Even prior to the greater governance established in the 1980s (see above). Formula 1 never introduced more than three circuits during a season. In the modern era of Formula 1, new circuits are somewhat sparse and never exceed more than two races during a season, even as the total number of circuits climbs.

In [7]:
#create our total circuit count by year as df1
plt_df1 = races.groupby(['year']).agg(circuit_count = ('circuitId', 'nunique')).reset_index()
#identify new circuit debuts by finding the min year per circuit then aggregating by year
plt_df2=races.groupby(['circuitId']).agg(min_year = ('year', 'min')).reset_index()\
  .groupby(['min_year']).agg(circuit_debut_count = ('circuitId', 'count')).reset_index()

plt_df1 = plt_df1.merge(plt_df2, how = 'left', left_on = 'year',right_on='min_year')

# Base chart
base = alt.Chart(plt_df1).encode(x=alt.X('year:O', title='Year'))

# Bar for debut circuits (F1 Red)
bar = base.mark_bar(color=f1_color_palette["Soft Red"]).encode(
    y=alt.Y('circuit_debut_count:Q',
            axis=alt.Axis(title='Debut Circuits',
                          titleColor=f1_color_palette["Soft Red"],
                          labelColor=f1_color_palette["Soft Red"]))
)

# Line for total circuits
line = base.mark_line(color=f1_color_palette["Track Asphalt"], strokeDash=[5, 2]).encode(
    y=alt.Y('circuit_count:Q',
            axis=alt.Axis(title='Total Circuits',
                          titleColor=f1_color_palette["Track Asphalt"],
                          labelColor=f1_color_palette["Track Asphalt"]))
)

# Combine
chart = alt.layer(bar, line).resolve_scale(y='independent').properties(
    title='F1 Circuit Debuts vs Total Active Circuits',
    height=300,
    width=800
)

chart

alt.LayerChart(...)

In [8]:
# as a follow up, I thought it would be valuable to visualize the proportion of new circuits over time

#create our total circuit count by year as df1
plt_df1 = races.groupby(['year']).agg(circuit_count = ('circuitId', 'nunique')).reset_index()
#identify new circuit debuts by finding the min year per circuit then aggregating by year
plt_df2=races.groupby(['circuitId']).agg(min_year = ('year', 'min')).reset_index()\
  .groupby(['min_year']).agg(circuit_debut_count = ('circuitId', 'count')).reset_index()

plt_df = plt_df1.merge(plt_df2, how = 'left', left_on = 'year',right_on='min_year')
plt_df = plt_df.fillna(0)\
  .assign(returning_circuits = lambda x: x.circuit_count - x.circuit_debut_count.fillna(0))\
  .melt(id_vars='year',value_vars=['circuit_debut_count','returning_circuits'])
plt_df['circuit_types'] = plt_df['variable'].replace({'circuit_debut_count': 'New Circuits', 'returning_circuits': 'Returning Circuits'})

plt_palette = {'New Circuits':f1_color_palette["Soft Red"],
               'Returning Circuits':f1_color_palette['Track Asphalt']}

fig = px.bar(plt_df,x='year',y='value',color='circuit_types',barmode='stack',
             template='simple_white',
             color_discrete_map=plt_palette)
fig.update_layout(bargap=0,
                  title = 'F1 Circuit Debuts vs Total Active Circuits',
                  xaxis_title = 'Year',
                  yaxis_title = 'Number of Circuits',
                  legend_title = 'Circuit Types',
                  legend=dict(
                      yanchor='top',
                      y=1,
                      xanchor='left',
                      x=.05
                  )
                  )
fig.show()


## Team Count per Season
Unlike race count, constructor count gradually declined to the current 10-team format. This may suggest a progressive consolidation to the minimum level of competition required to sustain the sport. Stated differently, at any given moment in F1 history, the current constructor group has a vested interest in preventing dilution of revenue generated by the sport. However, further consolidation beyond a 10 team minimum would likely harm the entertainment value by diminshing competition. This hypothesis suggests that 10 is a lower limit or carrying capacity that modern F1 governance structure progresses to but cannot excede.

In [9]:
plt_df = combined_results_df.groupby(['year']).agg(
    team_count = ('constructorId', 'nunique')
  ).reset_index()

fig = px.line(plt_df,x='year',y='team_count', title = 'Number of Constructors (Teams) per Season',markers='.',
              height =500,width=1000,template='seaborn')
#clean up the axis labels
fig.update_layout(xaxis_title = 'Season', yaxis_title = 'Number of Races')
fig.update_yaxes(rangemode='tozero')  # Ensures the y-axis starts at 0

fig.add_hline(y=10,
              annotation_text="Hypothesized Minimum Necessary Level of Competition",
              line_width=2,
              line_dash='dash',
              annotation_position = 'bottom left')

fig.show()

# Constructors & Drivers: Historical Performance
- Wins by Constructor Over Time
- Most Successful Drivers by Wins/Podiums/Poles vs Teammate

# Qualifying vs Finish Dynamics
-  Grid Position vs Finishing Position: To what extent does the qualifying result determine the race result?
- Track Dynamics: Which tracks are most determined by qualifying?
- Driver Dynamics: Which drivers are most capable of improving their results on race day?

# Summary Findings & Executive Summary
 - Key insights
 - Limitations & Areas of Future Analysis
 - Alternative Visualization Techniques

## Limitations & Areas of Future Analysis

- Source data set did not include any contextual or financial information (e.g., worldwide viewership or revenue). This data is necessary to understand the economic forces that influence the history of Formula 1. Future areas of analysis should explore these topics in conjunction with the analysis above.

## Alternative Visuals

## Alt Time Series Options
Ultimately a line chart was chosen for most time series visualizations due to simplicity and clarity. Other options such as bar chart and area chart were considered (see below):

 - Clear alignment of annotations with exact years, making it easier to associate key governance events with specific inflection points in performance.

 - Better readability over long time spans, especially when data spans multiple decades with relatively small year-over-year changes.

 - Reduced visual clutter, since area and bar charts can obscure baseline values or crowd the horizontal axis when combined with multiple annotation layers.

In [10]:
plt_df = combined_results_df.groupby(['year']).agg(
    team_count = ('constructorId', 'nunique')
  ).reset_index()

fig = px.bar(plt_df,x='year',y='team_count', title = 'Number of Constructors (Teams) per Season',
              height =500,width=1000,template='seaborn')
#clean up the axis labels
fig.update_layout(xaxis_title = 'Season', yaxis_title = 'Number of Races')
fig.show()

In [11]:
plt_df = combined_results_df.groupby(['year']).agg(
    team_count = ('constructorId', 'nunique')
  ).reset_index()

fig = px.area(plt_df,x='year',y='team_count', title = 'Number of Constructors (Teams) per Season',
              height =500,width=1000,template='seaborn')
#clean up the axis labels
fig.update_layout(xaxis_title = 'Season', yaxis_title = 'Number of Races')
fig.show()